Notebook will be run on kaggle non interactive session

In [ ]:
import pandas as pd
import numpy as np
import scipy.sparse
import gc
import optuna

# Scikit-learn, Boosters, TF, Transformers, FAISS...
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import lightgbm as lgb
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling1D, Dense, Concatenate, Dropout, Bidirectional, LSTM, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sentence_transformers import SentenceTransformer
import faiss

In [ ]:
class CFG:
    N_SPLITS = 5
    RANDOM_STATE = 42
    DATA_PATH = '/kaggle/input/amlc2025-dataset/train.csv'
    
    # --- Text Embedding Models ---
    ST_MODELS = [
        'sentence-transformers/all-MiniLM-L12-v2',
        'BAAI/bge-base-en-v1.5',
        'sentence-transformers/all-distilroberta-v1',
        'sentence-transformers/paraphrase-mpnet-base-v2'
    ]
    BGE_MODEL_INDEX = 1
    
    # --- Feature Generation Config ---
    KNN_N_NEIGHBORS = 10
    
    # --- Deep Learning Model Config ---
    DL_VOCAB_SIZE = 30000
    DL_MAX_LEN = 60

    # --- Default Hyperparameters (Placeholders for Tuning) ---
    LGBM_PARAMS = {'objective': 'regression_l1', 'metric': 'mae', 'n_estimators': 2000, 'learning_rate': 0.05, 'feature_fraction': 0.8, 'bagging_fraction': 0.8, 'num_leaves': 31, 'verbose': -1, 'n_jobs': -1, 'seed': 42}
    XGB_PARAMS = {'objective': 'reg:squarederror', 'eval_metric': 'mae', 'n_estimators': 2000, 'learning_rate': 0.05, 'max_depth': 7, 'subsample': 0.8, 'colsample_bytree': 0.8, 'tree_method': 'gpu_hist', 'seed': 42}
    LSTM_ATT_PARAMS = {'embedding_dim': 128, 'lstm_units': 64, 'dense_units': 64, 'dropout_rate': 0.3, 'learning_rate': 0.001}
    CNN_MULTI_PARAMS = {'embedding_dim': 128, 'cnn_filters': 64, 'dense_units': 128, 'dropout_rate': 0.5, 'learning_rate': 0.001}
    NN_MULTI_INPUT_PARAMS = {'embedding_dim': 64, 'gru_units': 64, 'text_branch_dense_units': 32, 'tabular_branch_dense_units': 32, 'final_dense_units': 64, 'dropout_rate': 0.4, 'learning_rate': 0.001}

In [ ]:
# Define the SMAPE metric function
def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    ratio = np.where(denominator == 0, 0, numerator / denominator)
    return np.mean(ratio) * 100

In [ ]:
df = pd.read_csv(CFG.DATA_PATH)
y = df['price'].values

In [ ]:
def create_extreme_engineered_features(df):
    df_out = pd.DataFrame(index=df.index)
    text_col = 'catalog_content'
    
    # Meta Features
    df_out[f'{text_col}_length'] = df[text_col].str.len()
    df_out[f'{text_col}_word_count'] = df[text_col].str.split().str.len()
    df_out[f'{text_col}_capital_ratio'] = df[text_col].apply(lambda t: sum(1 for c in t if c.isupper()) / (len(t) + 1e-9))
    
    # Entity Extraction (Example lists - expand these!)
    BRANDS = ['sony', 'samsung', 'nike', 'apple', 'kitchenaid']
    MATERIALS = ['cotton', 'leather', 'silk', 'gold', 'nylon', 'polyester', 'silver']
    
    text_lower = df[text_col].str.lower()
    df_out['has_brand'] = text_lower.apply(lambda t: any(brand in t for brand in BRANDS)).astype(int)
    for material in MATERIALS:
        df_out[f'has_{material}'] = text_lower.str.contains(material).astype(int)
        
    # Regex Features
    df_out['extracted_inch'] = text_lower.str.extract(r'(\d+\.?\d*)\s*(inch|\"|in\b)').iloc[:, 0].astype(float)
    df_out['extracted_gb'] = text_lower.str.extract(r'(\d+)\s*gb').iloc[:, 0].astype(float)
    
    # Fill NaNs from regex with a neutral value like 0
    return df_out.fillna(0)

In [ ]:
def generate_transformer_embeddings(df, models):
    all_embeddings = []
    for model_name in models:
        print(f"   Generating embeddings for: {model_name}")
        model = SentenceTransformer(model_name)
        embeddings = model.encode(df['catalog_content'].tolist(), show_progress_bar=True)
        all_embeddings.append(embeddings)
    return np.concatenate(all_embeddings, axis=1)

In [ ]:
def generate_knn_features(index_embeds, y_index, query_embeds, k):
    """
    Generates KNN features.
    'index_embeds' are used to build the search space.
    'query_embeds' are the items we want to find neighbors for.
    """
    d = index_embeds.shape[1]
    index = faiss.IndexFlatL2(d)
    
    # Check if GPU is available
    try:
        res = faiss.StandardGpuResources()
        gpu_index = faiss.index_cpu_to_gpu(res, 0, index)
        gpu_index.add(index_embeds.astype(np.float32))
        search_index = gpu_index
    except AttributeError:
        # Fallback to CPU if faiss-gpu is not installed or fails
        print("   FAISS-GPU not found or failed, falling back to CPU.")
        index.add(index_embeds.astype(np.float32))
        search_index = index

    # If query is the same as index, we are doing self-search. K+1 and ignore first result.
    is_self_search = np.array_equal(index_embeds, query_embeds)
    if is_self_search:
        _, I = search_index.search(query_embeds.astype(np.float32), k + 1)
        I = I[:, 1:] # Exclude the first column which is the item itself
    else:
        _, I = search_index.search(query_embeds.astype(np.float32), k)
    
    neighbor_prices = y_index[I]
    
    knn_feats = np.zeros((len(query_embeds), 3))
    knn_feats[:, 0] = np.mean(neighbor_prices, axis=1)
    knn_feats[:, 1] = np.median(neighbor_prices, axis=1)
    knn_feats[:, 2] = np.std(neighbor_prices, axis=1)
    
    return knn_feats


In [ ]:
print("Generating all features...")
extreme_engineered_features = create_extreme_engineered_features(df)
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=50000)
tfidf_features = tfidf_vectorizer.fit_transform(df['catalog_content'])
combined_embeddings = generate_transformer_embeddings(df, CFG.ST_MODELS)

In [ ]:
print("Splitting data to get a single fold for tuning...")
kf = KFold(n_splits=CFG.N_SPLITS, shuffle=True, random_state=CFG.RANDOM_STATE)
train_idx, val_idx = next(iter(kf.split(df))) # Get the first fold

In [ ]:
# Split all feature sets
X_train_text, X_val_text = df['catalog_content'].iloc[train_idx], df['catalog_content'].iloc[val_idx]
y_train, y_val = y[train_idx], y[val_idx]
X_train_eng, X_val_eng = extreme_engineered_features.iloc[train_idx].values, extreme_engineered_features.iloc[val_idx].values
X_train_tfidf, X_val_tfidf = tfidf_features[train_idx], tfidf_features[val_idx]
X_train_embed, X_val_embed = combined_embeddings[train_idx], combined_embeddings[val_idx]

In [ ]:
# Generate KNN features for the tuning fold
model_dims = [SentenceTransformer(m).get_sentence_embedding_dimension() for m in CFG.ST_MODELS]
bge_start_index = sum(model_dims[:CFG.BGE_MODEL_INDEX])
bge_end_index = bge_start_index + model_dims[CFG.BGE_MODEL_INDEX]


In [ ]:
bge_train_embeds = X_train_embed[:, bge_start_index:bge_end_index]
bge_val_embeds = X_val_embed[:, bge_start_index:bge_end_index]
knn_feats_train = generate_knn_features(bge_train_embeds, y_train, bge_train_embeds, CFG.KNN_N_NEIGHBORS)
knn_feats_val = generate_knn_features(bge_train_embeds, y_train, bge_val_embeds, CFG.KNN_N_NEIGHBORS)

print("\n--- COMMON SETUP COMPLETE. ALL DATA FOR TUNING IS READY. ---")

In [ ]:
# --- Assemble features for XGBoost ---
X_train_xgb = scipy.sparse.hstack((X_train_tfidf, X_train_eng))
X_val_xgb = scipy.sparse.hstack((X_val_tfidf, X_val_eng))

def xgb_objective(trial):
    params = {
        'objective': 'reg:squarederror', 'eval_metric': 'mae', 'n_estimators': 2000,
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'tree_method': 'gpu_hist', 'seed': 42
    }
    model = xgb.XGBRegressor(**params)
    model.fit(X_train_xgb, y_train, eval_set=[(X_val_xgb, y_val)], early_stopping_rounds=100, verbose=False)
    preds = model.predict(X_val_xgb)
    return smape(y_val, preds)

study = optuna.create_study(direction='minimize')
study.optimize(xgb_objective, n_trials=50)
print("\nBest XGBoost Parameters:", study.best_params)